In [13]:
import pandas as pd
import numpy as np
import itertools
import os

# Utility Function Imports (only import those needed)
from utilities import *

# Algorithm Imports
from Strategies.benchmarks import *
from Strategies.follow_the_loser import *
from Strategies.follow_the_winner import *
from Strategies.pattern_matching import *
from Strategies.meta_learning import *

# -------------------------------
# PART 1: Parameter Tuning Setup
# -------------------------------
price_relative_df = pd.read_csv("..\\Data\\Price Relative Vectors\\price_relative_vectors.csv", index_col=0)
b = initialize_portfolio(price_relative_df.shape[1])
price_relative_vectors_array = price_relative_df.values
dates = price_relative_df.index

# -------------------------------
# Evaluation Function for RMR
# -------------------------------
def evaluate_rmr(b, price_relative_vectors, window_size, epsilon, eta, freq=252, risk_free_rate=0.05):
    """
    Runs the RMR (Robust Median Reversion) strategy with given parameters and returns performance metrics.
    
    Parameters:
        b: Initial portfolio (numpy array)
        price_relative_vectors: numpy array of price relative vectors (shape: [T, N])
        window_size: Window size for computing the L1-median prediction.
        epsilon: Threshold for triggering the update.
        eta: Learning rate multiplier to scale the update step.
        freq: Frequency for Sharpe ratio calculation (default 252).
        risk_free_rate: Risk free rate for Sharpe ratio calculation (default 0.0).
        
    Returns:
        A dictionary containing performance metrics.
    """
    # Run RMR strategy with given hyperparameters
    b_n = rmr(b, price_relative_vectors, window_size=window_size, epsilon=epsilon, eta=eta)
    
    # Final wealth and exponential growth rate
    final_wealth = calculate_cumulative_wealth(b_n, price_relative_vectors)
    n_periods = len(price_relative_vectors)
    exp_growth = calculate_exponential_growth_rate(final_wealth, n_periods)
    
    # Sharpe ratio: compute cumulative wealth over time, then periodic returns, and then Sharpe ratio
    cum_wealth = calculate_cumulative_wealth_over_time(b_n, price_relative_vectors)
    daily_returns = compute_periodic_returns(cum_wealth)
    sharpe = compute_sharpe_ratio(daily_returns, freq=freq, risk_free_rate=risk_free_rate)
    
    return {
        'algorithm': 'rmr',
        'window_size': window_size,
        'epsilon': epsilon,
        'eta': eta,
        'final_wealth': final_wealth,
        'exp_growth': exp_growth,
        'sharpe': sharpe
    }

# -------------------------------
# Generic Tuning Function
# -------------------------------
def tune_strategy(evaluation_func, b, price_relative_vectors, grid):
    """
    Generic tuning function that runs grid search using the provided evaluation function.
    
    Parameters:
        evaluation_func: Function to evaluate an algorithm. It should accept the initial portfolio,
                         price relative vectors, and keyword arguments corresponding to the grid keys.
        b: Initial portfolio.
        price_relative_vectors: Price relative matrix.
        grid: Dictionary of parameter names and lists of values.
    
    Returns:
        results_df: A DataFrame of all results.
        best_result: The best result (dictionary) based on Sharpe ratio.
    """
    param_names = list(grid.keys())
    param_values = list(grid.values())
    
    results = []
    for params in itertools.product(*param_values):
        # Create a dictionary of parameters for this run
        param_dict = dict(zip(param_names, params))
        # Call the evaluation function with the parameter values (unpack as keyword arguments)
        result = evaluation_func(b, price_relative_vectors, **param_dict)
        # Add grid parameters to the result (if not already included)
        result.update(param_dict)
        results.append(result)
    
    # Convert results to a DataFrame
    results_df = pd.DataFrame(results)
    # Sort by Sharpe ratio descending (assuming higher Sharpe is better)
    results_df.sort_values(by='sharpe', ascending=False, inplace=True)
    # Get best result as a dictionary
    best_result = results_df.iloc[0].to_dict()
    return results_df, best_result

# -------------------------------
# Define the grid for RMR tuning
# -------------------------------
rmr_grid = {
    'window_size': [7],
    'epsilon': [1.0],
    'eta': [10.0, 20.0, 30.0]
}

# -------------------------------
# Run the tuning for RMR
# -------------------------------
rmr_results_df, best_rmr = tune_strategy(evaluate_rmr, b, price_relative_vectors_array, rmr_grid)

# Save the results to a CSV for repeatability
rmr_results_df.to_csv("..\\Data\\Tuning Data\\rmr_tuning_results.csv", index=False)

# Display best result
print("=== Best RMR Tuning Result ===")
print(best_rmr)

# For debugging/inspection, display the full results DataFrame:
print("=== All RMR Tuning Results ===")
rmr_results_df


=== Best RMR Tuning Result ===
{'algorithm': 'rmr', 'window_size': 7, 'epsilon': 1.0, 'eta': 20.0, 'final_wealth': 11.604418142350905, 'exp_growth': 0.0008119860549647878, 'sharpe': 0.9249419001192871}
=== All RMR Tuning Results ===


,algorithm,window_size,epsilon,eta,final_wealth,exp_growth,sharpe
1,rmr,7,1.0,20.0,11.604418,0.000812,0.924942
2,rmr,7,1.0,30.0,10.533506,0.000780,0.917823
0,rmr,7,1.0,10.0,12.981326,0.000849,0.911290
